In [6]:
import sys
from PyQt5.QtWidgets import QApplication, QGraphicsScene, QGraphicsView, QGraphicsItem, QGraphicsPathItem
from PyQt5.QtCore import Qt, QPointF, QRectF, QLineF
from PyQt5.QtGui import QPainterPath, QPolygonF
from PyQt5 import QtGui

class PetriNetItem(QGraphicsItem):
    def __init__(self, x, y, width, height, color, item_type):
        super().__init__()

        self.setPos(x, y)
        self.width = width
        self.height = height
        self.color = color
        self.item_type = item_type
        self.arcs = []  # List to store connected arcs
        self.setFlag(QGraphicsItem.ItemIsMovable)
        self.setFlag(QGraphicsItem.ItemSendsGeometryChanges)

    def boundingRect(self):
        return QRectF(-self.width / 2, -self.height / 2, self.width, self.height)

    def paint(self, painter, option, widget=None):
        painter.setBrush(self.color)
        if self.item_type == "place":
            painter.drawEllipse(self.boundingRect())
        else:
            painter.drawRect(self.boundingRect())

    def itemChange(self, change, value):
        if change == QGraphicsItem.ItemPositionChange:
            for arc in self.arcs:
                arc.updatePath()
        return super().itemChange(change, value)

class PetriNetArc(QGraphicsPathItem):
    def __init__(self, source, target):
        super().__init__()

        self.source = source
        self.target = target
        self.arrow_size = 10

        self.updatePath()

    def updatePath(self):
        source_point = self.source.scenePos()
        target_point = self.target.scenePos()

        line = QLineF(source_point, target_point)

        path = QPainterPath()
        path.moveTo(line.p1())
        path.lineTo(line.p2())

        angle = line.angle()

        arrowP1 = line.p2() - QPointF(self.arrow_size, 0)
        arrowP2 = line.p2() - QPointF(self.arrow_size, self.arrow_size / 2)
        arrowP3 = line.p2() - QPointF(0, self.arrow_size / 2)

        arrow = QPolygonF([arrowP1, arrowP2, arrowP3])
        transform = QtGui.QTransform().rotate(angle)

        path.addPolygon(transform.map(arrow))
        self.setPath(path)

def main():
    app = QApplication(sys.argv)
    viewer = QGraphicsView()

    scene = QGraphicsScene()
    viewer.setScene(scene)

    places = []
    transitions = []

    def createPlace(x, y):
        place = PetriNetItem(x, y, 40, 40, Qt.blue, "place")
        places.append(place)
        scene.addItem(place)

    def createTransition(x, y):
        transition = PetriNetItem(x, y, 40, 40, Qt.red, "transition")
        transitions.append(transition)
        scene.addItem(transition)

    def createArc(source, target):
        arc = PetriNetArc(source, target)
        source.arcs.append(arc)
        target.arcs.append(arc)
        scene.addItem(arc)

    # Example: Create places, transitions, and arcs
    createPlace(100, 100)
    createTransition(200, 200)
    createPlace(300, 100)
    createArc(places[0], transitions[0])
    createArc(transitions[0], places[1])

    viewer.show()
    sys.exit(app.exec_())

if __name__ == "__main__":
    main()


NameError: name 'QtGui' is not defined